<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="blue"><b>
          Face Recognition Deep Learning with PyWren over IBM Cloud Functions</b></font></font></th>
</table>

This notebook contains steps and code to demonstrate how serverless computing can provide great benefit for AI data preprocessing. We demonstrate Face Recognition deep learning over Watson Machine Learning service, while letting IBM Cloud Function to do the data preparation phase. As we will show this makes an entire process up to 50 times faster comparing to running the same code without leveraging serverless computing.

Our notebook is based on a blog <a href="https://hackernoon.com/building-a-facial-recognition-pipeline-with-deep-learning-in-tensorflow-66e7645015b8" target="_blank" rel="noopener no referrer">Building a Facial Recognition Pipeline with Deep Learning in Tensorflow </a> written by Cole Murray who kindly allowed us to use code and text from his blog.

This notebook introduces commands for getting data, training_definition persistance to Watson Machine Learning repository, model training, deployment and scoring.

Some familiarity with Python is helpful. This notebook uses 

- Python 3 
- <a href="https://dataplatform.cloud.ibm.com/docs/content/analyze-data/environments-parent.html" target="_blank" rel="noopener no referrer">Watson Studio environments.</a>
- IBM Cloud Functions
- <a href="https://github.com/pywren/pywren-ibm-cloud" target="_blank" rel="noopener no referrer">PyWren for IBM Cloud</a>



## Learning goals

In this notebook, you will learn how to:

-  Work with Watson Machine Learning experiments to train Deep Learning models (Tensorflow)
-  Save trained models in the Watson Machine Learning repository
-  Deploy a trained model online and score
-  How IBM Cloud Functions can be used for data preparation phase
-  Value of PyWren for IBM Cloud


## Contents


<a id="setup"></a>
## <span style="color:blue">1. Set up related IBM Cloud Services</span>

Before you use the sample code in this notebook, you must setup Watson Machine Learning Service, IBM Cloud Object Storage and IBM Cloud Functions.


### 1.1 Create Watson Machine Learning Service

Create a <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance is <a href="https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html" target="_blank" rel="noopener no referrer">here</a>).

### 1.2 Create IBM Cloud Object Storage

Create a <a href="https://console.bluemix.net/catalog/infrastructure/cloud-object-storage" target="_blank" rel="noopener no referrer">Cloud Object Storage (COS)</a> instance (a lite plan is offered and information about how to order storage is <a href="https://console.bluemix.net/docs/services/cloud-object-storage/basics/order-storage.html#order-storage" target="_blank" rel="noopener no referrer">here</a>). <br/>**Note: When using Watson Studio, you already have a COS instance associated with the project you are running the notebook in.**

- Create new credentials with HMAC: 
    - Go to your COS dashboard.
    - In the **Service credentials** tab, click **New Credential+**.
    - Add the inline configuration parameter: {"HMAC":true}, click **Add**. (For more information, see <a href="https://console.bluemix.net/docs/services/cloud-object-storage/hmac/credentials.html#using-hmac-credentials" target="_blank" rel="noopener no referrer">HMAC</a>.)

    This configuration parameter adds the following section to the instance credentials, (for use later in this notebook):
    ```
      "cos_hmac_keys": {
            "access_key_id": "-------",
            "secret_access_key": "-------"
       }
    ```



### 1.3 Create IBM Cloud Functions account
Setup IBM Cloud Functions account as described here. Please follow all the steps and make sure you can run "Hello World" example based on Python code. This will assure your Cloud Functions service is running

## <span style="color:blue"> 2. Dependencies installation </span>

Install the needed libraries for the Face Recognition. 
"dlib" dependency need to be installed via new environment. Create new environment based on Python 3.5 and add dependency in the customizaion section, as follows

    dependencies:
     - dlib

In [ ]:
%%capture
!curl -fsSL "https://git.io/fhe9X" | sh
try:
    import pywren_ibm_cloud as pywren
except:
    !curl -fsSL "https://git.io/fhe9X" | sh
    import pywren_ibm_cloud as pywren
try:
    import cv2
except:
    !pip install --user opencv-contrib-python
try:
    from openface.align_dlib import AlignDlib
except:    
    !git clone https://github.com/cmusatyalab/openface.git
    !cd openface ; python setup.py install

from uuid import uuid4

## <span style="color:blue">3. Configuration </span>
This section explains how to configure services

### 3.1 COS Connection
You need obtain both crednetials to the Cloud Functions and COS

You can find COS credentials in your COS instance dashboard under the Service credentials tab.
Note: the HMAC key, described in set up the environment is included in these credentials.



In [ ]:
cos_credentials = {
  "apikey": "***",
  "cos_hmac_keys": {
    "access_key_id": "***",
    "secret_access_key": "***"
  },
  "endpoints": "https://cos-service.bluemix.net/endpoints",
  "iam_apikey_description": "Auto generated apikey during resource-key operation for Instance - crn:v1:bluemix:public:cloud-object-storage:global:a/07a95aa44e6124e8b320b70cf88033fa:876e5285-4bef-4cf3-a89b-595e19648c7c::",
  "iam_apikey_name": "auto-generated-apikey-19a79dae-6a58-4b4f-878f-6839b711523f",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/07a95aa44e6124e8b320b70cf88033fa::serviceid:ServiceId-3f2cccee-61ec-4147-8732-9f58479ba26a",
  "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:a/07a95aa44e6124e8b320b70cf88033fa:876e5285-4bef-4cf3-a89b-595e19648c7c::"
}

Define the endpoint.

To do this, go to the **Endpoint** tab in the COS instance's dashboard to get the endpoint information, then enter it in the cell below:

In [ ]:
# Define endpoint information.
service_endpoint = 'https://s3-api.us-geo.objectstorage.softlayer.net'

You also need the IBM Cloud authorization endpoint to be able to create COS resource object.

In [ ]:
# Define the authorization endpoint.
auth_endpoint = 'https://iam.bluemix.net/oidc/token'

Install the boto library. This library allows Python developers to manage Cloud Object Storage (COS).

**Tip:** If `ibm_boto3` is not preinstalled in you environment, run the following command to install it: 

In [ ]:
# Run the command if ibm_boto3 is not installed.
%!pip install ibm-cos-sdk

In [ ]:
# Install the boto library.
import ibm_boto3
from ibm_botocore.client import Config

Create a Boto resource to be able to write data to COS.

In [ ]:
# Create a COS resource.
cos = ibm_boto3.resource('s3',
                         ibm_api_key_id=cos_credentials['apikey'],
                         ibm_service_instance_id=cos_credentials['resource_instance_id'],
                         ibm_auth_endpoint=auth_endpoint,
                         config=Config(signature_version='oauth'),
                         endpoint_url=service_endpoint)

### 3.2 IBM Cloud Functions setup

Obtain api key and endpoint to the IBM Cloud Functions service. Navigate the "API Key" menu and copy namespace, host and key. Make sure to add "https://" to the host

In [ ]:
config = {
          'ibm_cf':  {'endpoint': 'https://us-east.functions.cloud.ibm.com', 
                      'namespace': '<NAMESPACE>', 
                      'api_key': '<API KEY>'}, 
          'ibm_cos': {'endpoint': 'https://s3-api.us-geo.objectstorage.softlayer.net', 
                      'api_key' : '<API KEY>'},
           'pywren' : {'storage_bucket' : '<IBM COS BUCKET>'}
         }

PyWren engine requires it's server side component to be deplpoyed in advane. This step creates a new IBM Cloud Function function with PyWren server side runtime. This action will be used internally by PyWren during execution phases.

In [ ]:
from pywren_ibm_cloud.deployutil import clone_runtime
clone_runtime('cactusone/pywren:3.5', config, 'pywren-ibm-cloud-master')

## 4. Preprocessing Data using Dlib and Docker

### 3.1 Preparing the Data

You’ll use the LFW (Labeled Faces in the Wild) dataset as training data. Below are instructions how you can upload this dataset into your private COS bucket. Alternatively you may replace this with your dataset by following the same structure.


     Directory Structure
     ├── Tyra_Banks
     │ ├── Tyra_Banks_0001.jpg
     │ └── Tyra_Banks_0002.jpg
     ├── Tyron_Garner
     │ ├── Tyron_Garner_0001.jpg
     │ └── Tyron_Garner_0002.jpg

Create a COS bucket, which you will use to store the input data.

**Note:** The bucket names must be unique.

In [ ]:
bucket_uid = str(uuid4())
BUCKET = 'face-recognition-' + bucket_uid

if not cos.Bucket(BUCKET) in cos.buckets.all():
    print('Creating bucket "{}"...'.format(BUCKET))
    try:
        cos.create_bucket(Bucket=BUCKET)
    except ibm_boto3.exceptions.ibm_botocore.client.ClientError as e:
    print('Error: {}.'.format(e.response['Error']['Message']))

Now you should have 1 buckets.

In [ ]:
# Display a list of created buckets.
print(list(cos.buckets.all()))

The following step copies images from Labeled Faces in the Wild into your COS bucket.
We demonstrate with small data set of 14MB. If you wish to you use entire data set, then use 

    url = "http://vis-www.cs.umass.edu/lfw/lfw.tgz"

In [ ]:
import urllib.request
import tarfile
import io
 

def extractFromStream(url, cos, target_prefix = None):
    ftp_stream = urllib.request.urlopen(url)
    tarfile_like_object = io.BytesIO(ftp_stream.read())
    TarFile_object = tarfile.open(fileobj=tarfile_like_object)
    for member in TarFile_object:
        if member.isdir() == False:
            member_like_object = TarFile_object.extractfile(member)
            key = target_prefix
            if target_prefix is not None:
                key = target_prefix + '/' + member.name
            cos.Object(BUCKET, key).put(Body=member_like_object.read())
            break

 
 
url = "http://vis-www.cs.umass.edu/lfw/lfw-a.tgz"
extractFromStream(url, cos, "rawimages")


In [ ]:
for bucket_name in buckets:
    print(bucket_name)
    bucket_obj = cos.Bucket(bucket_name)
    for obj in bucket_obj.objects.all():
        print("  File: {}, {:4.2f}kB".format(obj.key, obj.size/1024))

### 3.2 Data preprocessing with serveless

Below, you’ll pre-process the images before passing them into the FaceNet model. Image pre-processing in a facial recognition context typically solves a few problems. These problems range from lighting differences, occlusion, alignment, segmentation. Below, you’ll address segmentation and alignment.
First, you’ll solve the segmentation problem by finding the largest face in an image. This is useful as our training data does not have to be cropped for a face ahead of time.
Second, you’ll solve alignment. In photographs, it is common for a face to not be perfectly center aligned with the image. To standardize input, you’ll apply a transform to center all images based on the location of eyes and bottom lip.


### 3.3 Detect, Crop & Align with Dlib

Upload dlib’s face landmark predictor into your COS bucket. You’ll use this face landmark predictor to find the location of the inner eyes and bottom lips of a face in an image. These coordinates will be used to center align the image.


In [ ]:
url = "http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"
extractFromStream(url, cos, 'predictor')

### 3.4 Preprocessing with IBM Cloud Functions
Next, you’ll create a preprocessor for your dataset. This file will read each image into memory, attempt to find the largest face, center align, and write the file to output. If a face cannot be found in the image, logging will be displayed to console with the filename.
As each image can be processed independently, python’s multiprocessing is used to process an image on each available cpu core.

In [ ]:
import logging
import os
import time
import pywren_ibm_cloud as pywren
import shutil
import cv2

from openface.align_dlib import AlignDlib
logger = logging.getLogger(__name__)

temp_dir = '/tmp'

def preprocess_image(bucket, key, data_stream, ibm_cos):
    """
    Detect face, align and crop :param input_path. Write output to :param output_path
    :param bucket: COS bucket
    :param key: COS key (object name ) - may contain delimiters
    :param storage_handler: can be used to read / write data from / into COS
    """
    crop_dim = 180
    print("Process bucket {} key {}".format(bucket, key))    
    # key of the form /subdir1/../subdirN/file_name
    key_components = key.split('/')
    file_name = key_components[len(key_components)-1]
    input_path = temp_dir + '/' + file_name
    if not os.path.exists(temp_dir + '/' + 'output'):
        os.makedirs(temp_dir + '/' +'output')
    output_path = temp_dir + '/' +'output/'  + file_name
    with open(input_path, 'wb') as localfile:
        shutil.copyfileobj(data_stream, localfile)
    exists = os.path.isfile(temp_dir + '/' +'shape_predictor_68_face_landmarks')
    if exists:
        pass;
    else:
        res = ibm_cos.get_object(Bucket = bucket, Key = 'predictor/shape_predictor_68_face_landmarks.dat')
        with open(temp_dir + '/' +'shape_predictor_68_face_landmarks', 'wb') as localfile:
            shutil.copyfileobj(res['Body'], localfile)
    align_dlib = AlignDlib(temp_dir + '/' +'shape_predictor_68_face_landmarks')
    image = _process_image(input_path, crop_dim, align_dlib)
    if image is not None:
        print('Writing processed file: {}'.format(output_path))
        cv2.imwrite(output_path, image)
        f = open(output_path, "rb")
        processed_image_path = os.path.join('output',key)
        ibm_cos.put_object(Bucket = bucket, Key = processed_image_path, Body = f)
        os.remove(output_path)
    else:
        print("Skipping filename: {}".format(input_path))
    os.remove(input_path)

def _process_image(filename, crop_dim, align_dlib):
    image = None
    aligned_image = None
    image = _buffer_image(filename)
    if image is not None:
        aligned_image = _align_image(image, crop_dim, align_dlib)
    else:
        raise IOError('Error buffering image: {}'.format(filename))
    return aligned_image

def _buffer_image(filename):
    logger.debug('Reading image: {}'.format(filename))
    image = cv2.imread(filename, )
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

def _align_image(image, crop_dim, align_dlib):
    bb = align_dlib.getLargestFaceBoundingBox(image)
    aligned = align_dlib.align(crop_dim, image, bb, landmarkIndices=AlignDlib.INNER_EYES_AND_BOTTOM_LIP)
    if aligned is not None:
        aligned = cv2.cvtColor(aligned, cv2.COLOR_BGR2RGB)
    return aligned

### 3.5 Getting Results

Now that you’ve created a pipeline, time to get results. As the script supports parallelism, you will see increased performance by running with multiple cores. You’ll need to run the preprocessor in the docker environment to have access to the installed libraries.
Below, you’ll mount your project directory as a volume inside the docker container and run the preprocessing script on your input data. The results will be written to a directory specified with command line arguments.

In [ ]:
raw_images = BUCKET + '/rawimages'    
pw = pywren.ibm_cf_executor(config=config, runtime='pywren-dlib-runtime_3.5')    
pw.map(preprocess_image, raw_images)
results = pw.get_result()
print("Execution completed")

### Review

Using Dlib, you detected the largest face in an image and aligned the center of the face by the inner eyes and bottom lip. This alignment is a method for standardizing each image for use as feature input.

### 1.3. Work with the WML service instance

From this point you need to adapt flows from the original blog to use Watson ML. Below is copy-paste from the original blog to easy the process

### Citations
Y. LeCun, L. Bottou, Y. Bengio, and P. Haffner. "Gradient-based learning applied to document recognition." Proceedings of the IEEE, 86(11):2278-2324, November 1998.

We follow section Creating Embeddings in Tensorflow from https://hackernoon.com/building-a-facial-recognition-pipeline-with-deep-learning-in-tensorflow-66e7645015b8

Assume that data was prepocessed and stored in COS.

    bucket name : gilvdata

all images stored under 
    output/lfw/test/images/*

for example

    output/lfw/test/images/AJ_Cook/AJ_Cook_0001.jpg
    output/lfw/test/images/AJ_Lamas/AJ_Lamas_0001.jpg
    output/lfw/test/images/Aaron_Eckhart/Aaron_Eckhart_0001.jpg
    output/lfw/test/images/Aaron_Guiel/Aaron_Guiel_0001.jpg
    output/lfw/test/images/Aaron_Patterson/Aaron_Patterson_0001.jpg
    output/lfw/test/images/Aaron_Peirsol/Aaron_Peirsol_0001.jpg
    output/lfw/test/images/Aaron_Peirsol/Aaron_Peirsol_0002.jpg

Model for dlib that was used to pre-process image is stored under

    lfw/model/shape_predictor_68_face_landmarks.dat

(this doesn't seems needed for the TensorFlow part of the blog )

## Creating Embeddings in Tensorflow

Now that you’ve preprocessed the data, you’ll generate vector embeddings of each identity. These embeddings can then be used as input to a classification, regression or clustering task.

### Download Weights

You’ll use the Inception Resnet V1 as your convolutional neural network. First, create a file to download the weights to the model.
By using pre-trained weights, you are able to apply transfer learning to a new dataset, in this tutorial the LFW dataset:

In [ ]:
import argparse

import logging
import os
import requests
import zipfile

"""
This file is copied from:
https://github.com/davidsandberg/facenet/blob/master/src/download_and_extract_model.py
"""

model_dict = {
    '20170511-185253': '0B5MzpY9kBtDVOTVnU3NIaUdySFE'
}


def download_and_extract_model(model_name, data_dir):
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    file_id = model_dict[model_name]
    destination = os.path.join(data_dir, model_name + '.zip')
    if not os.path.exists(destination):
        print('Downloading model to %s' % destination)
        download_file_from_google_drive(file_id, destination)
        with zipfile.ZipFile(destination, 'r') as zip_ref:
            print('Extracting model to %s' % data_dir)
            zip_ref.extractall(data_dir)


def download_file_from_google_drive(file_id, destination):
    URL = "https://drive.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params={'id': file_id}, stream=True)
    token = get_confirm_token(response)

    if token:
        params = {'id': file_id, 'confirm': token}
        response = session.get(URL, params=params, stream=True)

    save_response_content(response, destination)


def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None


def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk:  # filter out keep-alive new chunks
                f.write(chunk)


if __name__ == '__main__':
    logging.basicConfig(level=logging.DEBUG)
    parser = argparse.ArgumentParser(add_help=True)
    parser.add_argument('--model-dir', type=str, action='store', dest='model_dir',
                        help='Path to model protobuf graph')

    args = parser.parse_args()

    download_and_extract_model('20170511-185253', args.model_dir)

'''
$ docker run -v $PWD:/medium-facenet-tutorial \
-e PYTHONPATH=$PYTHONPATH:/medium-facenet-tutorial \
-it colemurray/medium-facenet-tutorial python3 /medium-facenet-tutorial/medium_facenet_tutorial/download_and_extract_model.py \
--model-dir /medium-facenet-tutorial/etc
'''

### Load Embeddings

Below, you’ll utilize Tensorflow’s queue api to load the preprocessed images in parallel. By using queues, images can be loaded in parallel using multi-threading. When using a GPU, this allows image preprocessing to be performed on CPU, while matrix multiplication is performed on GPU.

In [ ]:
import logging
import os

import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops

logger = logging.getLogger(__name__)


def read_data(image_paths, label_list, image_size, batch_size, max_nrof_epochs, num_threads, shuffle, random_flip,
              random_brightness, random_contrast):
    """
    Creates Tensorflow Queue to batch load images. Applies transformations to images as they are loaded.
    :param random_brightness: 
    :param random_flip: 
    :param image_paths: image paths to load
    :param label_list: class labels for image paths
    :param image_size: size to resize images to
    :param batch_size: num of images to load in batch
    :param max_nrof_epochs: total number of epochs to read through image list
    :param num_threads: num threads to use
    :param shuffle: Shuffle images
    :param random_flip: Random Flip image
    :param random_brightness: Apply random brightness transform to image
    :param random_contrast: Apply random contrast transform to image
    :return: images and labels of batch_size
    """

    images = ops.convert_to_tensor(image_paths, dtype=tf.string)
    labels = ops.convert_to_tensor(label_list, dtype=tf.int32)

    # Makes an input queue
    input_queue = tf.train.slice_input_producer((images, labels),
                                                num_epochs=max_nrof_epochs, shuffle=shuffle, )

    images_labels = []
    imgs = []
    lbls = []
    for _ in range(num_threads):
        image, label = read_image_from_disk(filename_to_label_tuple=input_queue)
        image = tf.random_crop(image, size=[image_size, image_size, 3])
        image.set_shape((image_size, image_size, 3))
        image = tf.image.per_image_standardization(image)

        if random_flip:
            image = tf.image.random_flip_left_right(image)

        if random_brightness:
            image = tf.image.random_brightness(image, max_delta=0.3)

        if random_contrast:
            image = tf.image.random_contrast(image, lower=0.2, upper=1.8)

        imgs.append(image)
        lbls.append(label)
        images_labels.append([image, label])

    image_batch, label_batch = tf.train.batch_join(images_labels,
                                                   batch_size=batch_size,
                                                   capacity=4 * num_threads,
                                                   enqueue_many=False,
                                                   allow_smaller_final_batch=True)
    return image_batch, label_batch


def read_image_from_disk(filename_to_label_tuple):
    """
    Consumes input tensor and loads image
    :param filename_to_label_tuple: 
    :type filename_to_label_tuple: list
    :return: tuple of image and label
    """
    label = filename_to_label_tuple[1]
    file_contents = tf.read_file(filename_to_label_tuple[0])
    example = tf.image.decode_jpeg(file_contents, channels=3)
    return example, label


def get_image_paths_and_labels(dataset):
    image_paths_flat = []
    labels_flat = []
    for i in range(int(len(dataset))):
        image_paths_flat += dataset[i].image_paths
        labels_flat += [i] * len(dataset[i].image_paths)
    return image_paths_flat, labels_flat


def get_dataset(input_directory):
    dataset = []

    classes = os.listdir(input_directory)
    classes.sort()
    nrof_classes = len(classes)
    for i in range(nrof_classes):
        class_name = classes[i]
        facedir = os.path.join(input_directory, class_name)
        if os.path.isdir(facedir):
            images = os.listdir(facedir)
            image_paths = [os.path.join(facedir, img) for img in images]
            dataset.append(ImageClass(class_name, image_paths))

    return dataset


def filter_dataset(dataset, min_images_per_label=10):
    filtered_dataset = []
    for i in range(len(dataset)):
        if len(dataset[i].image_paths) < min_images_per_label:
            logger.info('Skipping class: {}'.format(dataset[i].name))
            continue
        else:
            filtered_dataset.append(dataset[i])
    return filtered_dataset


def split_dataset(dataset, split_ratio=0.8):
    train_set = []
    test_set = []
    min_nrof_images = 2
    for cls in dataset:
        paths = cls.image_paths
        np.random.shuffle(paths)
        split = int(round(len(paths) * split_ratio))
        if split < min_nrof_images:
            continue  # Not enough images for test set. Skip class...
        train_set.append(ImageClass(cls.name, paths[0:split]))
        test_set.append(ImageClass(cls.name, paths[split:-1]))
    return train_set, test_set


class ImageClass():
    def __init__(self, name, image_paths):
        self.name = name
        self.image_paths = image_paths

    def __str__(self):
        return self.name + ', ' + str(len(self.image_paths)) + ' images'

    def __len__(self):
        return len(self.image_paths)



### Train a Classifier
With the input queue squared away, you’ll move on to creating the embeddings.
First, you’ll load the images from the queue you created. While training, you’ll apply preprocessing to the image. This preprocessing will add random transformations to the image, creating more images to train on.
These images will be fed in a batch size of 128 into the model. This model will return a 128 dimensional embedding for each image, returning a 128 x 128 matrix for each batch.
After these embeddings are created, you’ll use them as feature inputs into a scikit-learn’s SVM classifier to train on each identity. Identities with less than 10 images will be dropped. This parameter is tunable from command-line.

In [ ]:
import argparse
import logging
import os
import pickle
import sys
import time

import numpy as np
import tensorflow as tf
from sklearn.svm import SVC
from tensorflow.python.platform import gfile

from lfw_input import filter_dataset, split_dataset, get_dataset
from medium_facenet_tutorial import lfw_input

logger = logging.getLogger(__name__)


def main(input_directory, model_path, classifier_output_path, batch_size, num_threads, num_epochs,
         min_images_per_labels, split_ratio, is_train=True):
    """
    Loads images from :param input_dir, creates embeddings using a model defined at :param model_path, and trains
     a classifier outputted to :param output_path
     
    :param input_directory: Path to directory containing pre-processed images
    :param model_path: Path to protobuf graph file for facenet model
    :param classifier_output_path: Path to write pickled classifier
    :param batch_size: Batch size to create embeddings
    :param num_threads: Number of threads to utilize for queuing
    :param num_epochs: Number of epochs for each image
    :param min_images_per_labels: Minimum number of images per class
    :param split_ratio: Ratio to split train/test dataset
    :param is_train: bool denoting if training or evaluate
    """

    start_time = time.time()
    with tf.Session(config=tf.ConfigProto(log_device_placement=False)) as sess:
        train_set, test_set = _get_test_and_train_set(input_directory, min_num_images_per_label=min_images_per_labels,
                                                      split_ratio=split_ratio)
        if is_train:
            images, labels, class_names = _load_images_and_labels(train_set, image_size=160, batch_size=batch_size,
                                                                  num_threads=num_threads, num_epochs=num_epochs,
                                                                  random_flip=True, random_brightness=True,
                                                                  random_contrast=True)
        else:
            images, labels, class_names = _load_images_and_labels(test_set, image_size=160, batch_size=batch_size,
                                                                  num_threads=num_threads, num_epochs=1)

        _load_model(model_filepath=model_path)

        init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
        sess.run(init_op)

        images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
        embedding_layer = tf.get_default_graph().get_tensor_by_name("embeddings:0")
        phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")

        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord, sess=sess)

        emb_array, label_array = _create_embeddings(embedding_layer, images, labels, images_placeholder,
                                                    phase_train_placeholder, sess)

        coord.request_stop()
        coord.join(threads=threads)
        logger.info('Created {} embeddings'.format(len(emb_array)))

        classifier_filename = classifier_output_path

        if is_train:
            _train_and_save_classifier(emb_array, label_array, class_names, classifier_filename)
        else:
            _evaluate_classifier(emb_array, label_array, classifier_filename)

        logger.info('Completed in {} seconds'.format(time.time() - start_time))


def _get_test_and_train_set(input_dir, min_num_images_per_label, split_ratio=0.7):
    """
    Load train and test dataset. Classes with < :param min_num_images_per_label will be filtered out.
    :param input_dir: 
    :param min_num_images_per_label: 
    :param split_ratio: 
    :return: 
    """
    dataset = get_dataset(input_dir)
    dataset = filter_dataset(dataset, min_images_per_label=min_num_images_per_label)
    train_set, test_set = split_dataset(dataset, split_ratio=split_ratio)

    return train_set, test_set


def _load_images_and_labels(dataset, image_size, batch_size, num_threads, num_epochs, random_flip=False,
                            random_brightness=False, random_contrast=False):
    class_names = [cls.name for cls in dataset]
    image_paths, labels = lfw_input.get_image_paths_and_labels(dataset)
    images, labels = lfw_input.read_data(image_paths, labels, image_size, batch_size, num_epochs, num_threads,
                                         shuffle=False, random_flip=random_flip, random_brightness=random_brightness,
                                         random_contrast=random_contrast)
    return images, labels, class_names


def _load_model(model_filepath):
    """
    Load frozen protobuf graph
    :param model_filepath: Path to protobuf graph
    :type model_filepath: str
    """
    model_exp = os.path.expanduser(model_filepath)
    if os.path.isfile(model_exp):
        logging.info('Model filename: %s' % model_exp)
        with gfile.FastGFile(model_exp, 'rb') as f:
            graph_def = tf.GraphDef()
            graph_def.ParseFromString(f.read())
            tf.import_graph_def(graph_def, name='')
    else:
        logger.error('Missing model file. Exiting')
        sys.exit(-1)


def _create_embeddings(embedding_layer, images, labels, images_placeholder, phase_train_placeholder, sess):
    """
    Uses model to generate embeddings from :param images.
    :param embedding_layer: 
    :param images: 
    :param labels: 
    :param images_placeholder: 
    :param phase_train_placeholder: 
    :param sess: 
    :return: (tuple): image embeddings and labels
    """
    emb_array = None
    label_array = None
    try:
        i = 0
        while True:
            batch_images, batch_labels = sess.run([images, labels])
            logger.info('Processing iteration {} batch of size: {}'.format(i, len(batch_labels)))
            emb = sess.run(embedding_layer,
                           feed_dict={images_placeholder: batch_images, phase_train_placeholder: False})

            emb_array = np.concatenate([emb_array, emb]) if emb_array is not None else emb
            label_array = np.concatenate([label_array, batch_labels]) if label_array is not None else batch_labels
            i += 1

    except tf.errors.OutOfRangeError:
        pass

    return emb_array, label_array


def _train_and_save_classifier(emb_array, label_array, class_names, classifier_filename_exp):
    logger.info('Training Classifier')
    model = SVC(kernel='linear', probability=True, verbose=False)
    model.fit(emb_array, label_array)

    with open(classifier_filename_exp, 'wb') as outfile:
        pickle.dump((model, class_names), outfile)
    logging.info('Saved classifier model to file "%s"' % classifier_filename_exp)


def _evaluate_classifier(emb_array, label_array, classifier_filename):
    logger.info('Evaluating classifier on {} images'.format(len(emb_array)))
    if not os.path.exists(classifier_filename):
        raise ValueError('Pickled classifier not found, have you trained first?')

    with open(classifier_filename, 'rb') as f:
        model, class_names = pickle.load(f)

        predictions = model.predict_proba(emb_array, )
        best_class_indices = np.argmax(predictions, axis=1)
        best_class_probabilities = predictions[np.arange(len(best_class_indices)), best_class_indices]

        for i in range(len(best_class_indices)):
            print('%4d  %s: %.3f' % (i, class_names[best_class_indices[i]], best_class_probabilities[i]))

        accuracy = np.mean(np.equal(best_class_indices, label_array))
        print('Accuracy: %.3f' % accuracy)


if __name__ == '__main__':
    logging.basicConfig(level=logging.INFO)
    parser = argparse.ArgumentParser(add_help=True)
    parser.add_argument('--model-path', type=str, action='store', dest='model_path',
                        help='Path to model protobuf graph')
    parser.add_argument('--input-dir', type=str, action='store', dest='input_dir',
                        help='Input path of data to train on')
    parser.add_argument('--batch-size', type=int, action='store', dest='batch_size',
                        help='Input path of data to train on', default=128)
    parser.add_argument('--num-threads', type=int, action='store', dest='num_threads', default=16,
                        help='Number of threads to utilize for queue')
    parser.add_argument('--num-epochs', type=int, action='store', dest='num_epochs', default=3,
                        help='Path to output trained classifier model')
    parser.add_argument('--split-ratio', type=float, action='store', dest='split_ratio', default=0.7,
                        help='Ratio to split train/test dataset')
    parser.add_argument('--min-num-images-per-class', type=int, action='store', default=10,
                        dest='min_images_per_class', help='Minimum number of images per class')
    parser.add_argument('--classifier-path', type=str, action='store', dest='classifier_path',
                        help='Path to output trained classifier model')
    parser.add_argument('--is-train', action='store_true', dest='is_train', default=False,
                        help='Flag to determine if train or evaluate')

    args = parser.parse_args()

    main(input_directory=args.input_dir, model_path=args.model_path, classifier_output_path=args.classifier_path,
         batch_size=args.batch_size, num_threads=args.num_threads, num_epochs=args.num_epochs,
         min_images_per_labels=args.min_images_per_class, split_ratio=args.split_ratio, is_train=args.is_train)
'''
$ docker run -v $PWD:/medium-facenet-tutorial \
-e PYTHONPATH=$PYTHONPATH:/medium-facenet-tutorial \
-it colemurray/medium-facenet-tutorial \
python3 /medium-facenet-tutorial/medium_facenet_tutorial/train_classifier.py \
--input-dir /medium-facenet-tutorial/output/intermediate \
--model-path /medium-facenet-tutorial/etc/20170511-185253/20170511-185253.pb \
--classifier-path /medium-facenet-tutorial/output/classifier.pkl \
--num-threads 16 \
--num-epochs 25 \
--min-num-images-per-class 10 \
--is-train 
# ~16 mins to complete on MBP
'''

Evaluating the Results
Now that you’ve trained the classifier, you’ll feed it new images it has not trained on. You’ll remove the is_train flag from the previous command to evaluate your results.

    docker run -v $PWD:/$(basename $PWD) \
    -e PYTHONPATH=$PYTHONPATH:/medium-facenet-tutorial \
    -it colemurray/medium-facenet-tutorial \
    python3 /medium-facenet-tutorial/medium_facenet_tutorial/train_classifier.py \
    --input-dir /medium-facenet-tutorial/output/intermediate \
    --model-path /medium-facenet-tutorial/etc/20170511-185253/20170511-185253.pb \
    --classifier-path /medium-facenet-tutorial/output/classifier.pkl \
    --num-threads 16 \
    --num-epochs 5 \
    --min-num-images-per-class 10
    
After inference is on each image is complete, you’ll see results printed to console. At 5 epochs, you’ll see ~85.0% accuracy. Training @ 25 epochs gave results:

### Authors

**Gil Vernik? Who else? **

Copyright © 2019 IBM. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>